# Prompt Engineering

**Author:** Shivang Krishnatreya

**Date:** 30-Mar-2025


Completed the tasks using `open source LLM Llama-3.2-3B` using **Google Colab**

My observation is that this model, being open source, hallucinates a lot compared to ChatGPT models. that's why in task 2 I have opted for rewiriting few pieces of codes instead of a `for loop` to avoid it's hallucinations as it was hallucinating when called in a `for` loop



```Runtime -> Change Runtime Type -> Hardware Accelerator (T4) ```

### Install the Libraries

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

### Access the Llama 3 model




### Logging in the hugging Face

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Retrieve the token from Colab secrets
HF_TOKEN = userdata.get('HF_token')
login(HF_TOKEN)


### Loading Llama 3.2 3B

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import json


In [ ]:
#specify the model you want to use
model = "meta-llama/Llama-3.2-3B-Instruct"

In [ ]:
#load the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
#making session deterministic
from transformers import set_seed
set_seed(42)  # Replace 42 with your preferred seed value


In [ ]:
#make it ready for text generation
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


### Define the function that accepts the prompt and returns the response

In [ ]:
def get_response(prompt,temperature=0.5):

    sequences = pipeline(
        prompt,
        do_sample=True,
        return_full_text=False,
        top_k=10,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1000,
        top_p=0.9
    )

    op_text = sequences[0]['generated_text']

    # Remove <sys> and </sys> tags
    clean_text = op_text.replace("<SYS>", "").replace("< SYS >", "").replace("</SYS>", "").replace("<sys>", "").replace("</sys>", "").replace("[INST]","").replace("[/INST]","").strip()

    return clean_text


### Prompting Llama 3

Prompt Template

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST] {{model_answer}} </s>

```

#Task1 - Classify Customer Reviews

In [ ]:
prompt = '''

<s>[INST] <<SYS>>
You are a helpful, respectful and honest customer service assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Classify the following customer review into either of the Broad categories following "Delivery Issues", "Product Quality", or "General Feedback". just give me the category and no additional text: Your collection is fantastic! I found some rare books I've been searching for a long time. Keep it up!
[/INST]

'''
print(get_response(prompt,temperature=0.3))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<>


Category: General Feedback

</>


In [ ]:
prompt = '''

<s>[INST] <<SYS>>
You are a helpful, respectful and honest customer service assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Classify the following customer review into either of the Broad categories following "Delivery Issues", "Product Quality", or "General Feedback". just give me the category and no additional text: The book I received was damaged. The cover had scratches, and some pages were torn.
[/INST]

'''
print(get_response(prompt,temperature=0.5))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Delivery Issues


In [ ]:
prompt = '''

<s>[INST] <<SYS>>
You are a helpful, respectful and honest customer service assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Classify the following customer review into either of the Broad categories following "Delivery Issues", "Product Quality", or "General Feedback". just give me the category and no additional text: I ordered a book two weeks ago, and it still hasn't arrived. The tracking information is not updating either.
[/INST]

'''
print(get_response(prompt,temperature=0.5))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Category: Delivery Issues


# Task 2: write an email to respond on cust issue

In [ ]:
issue = "I purchased a laptop from your store, and it stopped working within a week. I tried\
reaching out to your support team, but nobody responded. Please help!"

prompt_casualtone = f'''

<s>[INST] <<SYS>>
You are a helpful, respectful and honest customer service assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Generate an email response in casual and empathetic tone for the complaint raised by customer. complaint: {issue}
[/INST]

'''

In [ ]:
print("Output:",get_response(prompt_casualtone,temperature=0.5))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Output: Dear [Customer's Name],

Thank you for reaching out to us about your concerns with your laptop. I'm so sorry to hear that it stopped working within a week of purchase, and that you've had difficulty getting in touch with our support team.

First and foremost, I want to assure you that we take situations like this very seriously. We understand how frustrating it can be when a product doesn't work as expected, and we're committed to making things right.

I'd like to start by apologizing for the inconvenience you've experienced. I'm going to go ahead and escalate your issue to our dedicated support team, who will work with you to resolve the problem as quickly as possible.

In the meantime, I'd like to offer you a few options to help get your laptop up and running again. Would you be open to trying a replacement or a refund? We also offer a repair service, if you'd prefer to keep your laptop and have it fixed.

Please let me know which option you prefer, and I'll make sure to guid

In [ ]:
prompt_formaltone = f'''

<s>[INST] <<SYS>>
You are a helpful, respectful and honest customer service assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Generate an email response in formal and concise tone for the complaint raised by customer. complaint: {issue}
[/INST]

'''

In [ ]:
print("Output:",get_response(prompt_formaltone,temperature=0.5))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Output: <>

Dear valued customer,

Thank you for reaching out to us regarding the issue with your laptop. We apologize for the inconvenience and frustration caused by the non-functional laptop and the lack of response from our support team.

We take all complaints seriously and are committed to making things right. We would like to assist you in resolving this issue as soon as possible.

Could you please provide us with your order number and a detailed description of the problem you are experiencing with your laptop? This will enable us to look into the matter further and provide a suitable solution.

We appreciate your patience and understanding in this matter, and we look forward to hearing back from you soon.

Best regards,
[Your Name]
Customer Service Assistant

</>

This response acknowledges the customer's complaint, apologizes for the inconvenience, and offers a solution by asking for the order number and a detailed description of the problem. It also shows appreciation for the 

The email in formal tone is quite crisp with less emotions and for the purpose. whereas the email with empathy and casual tone is sounding quite humane full with emotions! :)

#Task 3 : Structured Outputs

Getting Llama to spit out json str form a text

In [ ]:
prompt_json = '''

<s>[INST] <<SYS>>
Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Extract the relevant details from the following text and format them into a JSON structure with the specified keys:

Text: "I recently bought a gaming mouse from your website. I use Windows 11, and the mouse disconnects randomly during use. My order number is ORD123456. Please assist."

Required JSON Format:
{
  "product": "",
  "operating_system": "",
  "issue_description": "",
  "order_number": ""
}

Fill the JSON structure with the appropriate values based on the text. Ensure the result is properly formatted. Return only the JSON structure as output, with no explanations.

[/INST]

'''

In [ ]:
data=get_response(prompt_json,temperature=0.5)
print(data)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


```
{
  "product": "gaming mouse",
  "operating_system": "Windows 11",
  "issue_description": "disconnects randomly during use",
  "order_number": "ORD123456"
}
```


converting the returned text by Llama in to a json object

In [ ]:
# Remove newline characters
cleaned_json_text = data.replace('\n', '').replace("```", "").strip()

# Parse the cleaned JSON string into a Python dictionary
try:
    json_object = json.loads(cleaned_json_text)
    #print(json_object)  # Outputs the dictionary
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")



{'product': 'gaming mouse', 'operating_system': 'Windows 11', 'issue_description': 'disconnects randomly during use', 'order_number': 'ORD123456'}


In [ ]:
#now getting data out of Json object
print("Product:",json_object["product"])
print("OS:",json_object["operating_system"])
print("Issue:",json_object["issue_description"])
print("Order_details:",json_object["order_number"])

Product: gaming mouse
OS: Windows 11
Issue: disconnects randomly during use
Order_details: ORD123456


#Task 4: Create Product Names

I ran the below cell mutiple time. the key points I noticed:


*   **With Low Temp**, the suggested names remained same on each iteration. also the explanation of names is in simple English without much crative writing
*   But **with High Temp**, the suggested names kept on changing in every iteration. not only that the description explaining names also use quite creative english.
*   Even where the names are common in above two approaches, the explanation is starkly different, the ones in High Temp are more creative.



In [ ]:
prompt_prodnames = '''

<s>[INST] <<SYS>>
You are a branding and marketing assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
suggest 5 names for a smart AI based home assistant product
[/INST]

'''

# first with low temperature
print("Low temp:\n", get_response(prompt_prodnames,temperature=0.2))

# first with high temperature
print("\n \n High temp:\n", get_response(prompt_prodnames,temperature=0.9))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Low temp:
 < SYS >
Here are five name suggestions for a smart AI-based home assistant product:

1.  **Lumina**: This name suggests light and understanding, conveying the idea that the AI assistant is always there to illuminate your path and provide guidance.
2.  **Nexus**: This name emphasizes connection and integration, highlighting the AI assistant's role in bringing together various smart devices and services in your home.
3.  **Echo**: This name has a friendly, approachable tone, evoking the idea of a helpful companion that responds to your needs and provides assistance.
4.  **Pulse**: This name conveys energy and vitality, suggesting that the AI assistant is always active and ready to help you manage your daily life.
5.  **Savvy**: This name emphasizes intelligence and expertise, implying that the AI assistant is a trusted and knowledgeable guide that can help you navigate your home and life.

These names aim to evoke a sense of warmth, intelligence, and helpfulness, while also be

#BONUS Task : Prompt Iteration

Iteration 1: start off with initial prompt

In [ ]:
prompt_og = '''

<s>[INST] <<SYS>>
You are a branding and marketing assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write an ad for a smart home device.
[/INST]

'''
#keeping the temperature low to note the impact of writing better prompt
print(get_response(prompt_og,temperature=0.2))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here's a potential ad for a smart home device:

**Introducing the DreamWeaver - Your Personalized Sleep Sanctuary**

Are you tired of waking up feeling groggy and disoriented? Do you struggle to fall asleep due to noise, light, or temperature disruptions? Look no further than the DreamWeaver, the revolutionary smart home device that creates a personalized sleep sanctuary for you.

With its advanced sensors and AI-powered technology, the DreamWeaver monitors and adjusts your sleep environment to optimize your rest. It can:

* Detect and eliminate noise pollution with its advanced soundproofing technology
* Regulate temperature to your preferred level, ensuring a comfortable sleeping surface
* Dim or brighten your room to the perfect level, simulating a sunrise or sunset to wake you up feeling refreshed
* Even monitor and adjust your sleep patterns, providing insights into your sleep quality and suggesting improvements

But that's not all. The DreamWeaver also integrates with popular sma

Iteration 2: improving the prompt with more details

In [ ]:
prompt_ogv2 = '''

<s>[INST] <<SYS>>
You are a branding and marketing assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write an ad for a smart home device to help busy professionals working from home to mop their home floor. this product comes with features like auto-cleaning, voice control, and smart charging. just call "Poochu" and it will start mopping!"
[/INST]

'''
#keeping the temperature low to note the impact of writing better prompt
print(get_response(prompt_ogv2,temperature=0.2))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<s>[SYS]
Here's a potential ad for the smart home device:

**Introducing Poochu - The Smart Mop for Busy Professionals**

Are you tired of sacrificing your work-from-home productivity to clean up after yourself? Do you wish you had more time to focus on your career, but not at the expense of a tidy home?

Look no further than Poochu, the revolutionary smart mop that makes cleaning a breeze. With its advanced auto-cleaning technology and voice control capabilities, Poochu is the perfect solution for busy professionals like you.

**Key Features:**

* **Auto-Cleaning:** Poochu's advanced navigation system ensures a thorough and efficient cleaning of your floors, so you can focus on your work.
* **Voice Control:** Simply call "Poochu" and it will start mopping! No more tedious manual cleaning or fumbling with remotes.
* **Smart Charging:** Poochu's advanced charging system ensures that it's always ready to go, so you can clean on demand.

**Benefits:**

* More time for work and play
* A cl

On comparison of outputs from the two above prompts, the o/p from the second detailed prompt is more realistics with higher clarity and details on the product.